![HydroSAR Banner](./NotebookAddOns/HydroSARbanner.jpg)

# Download HAND from GEE

## Download raster tiles from HAND dataset on Google Earth Engine

**Part of NASA A.37 Project: Integrating SAR Data for Improved Resilience and Response to Weather-Related Disasters**

### PI: Franz J. Meyer
**Version 0.1.3 - 2021/01/24**

Change Log: See bottom of the notebook.

Contact: **batuhan.osmanoglu@nasa.gov**

In [23]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

URLWidget()

In [24]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/hydrosar':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "hydrosar" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select "hydrosar" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "hydrosar" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

---

## 0. Importing Relevant Python Packages

The first step in any notebook is to import the required Python libraries into the Jupyter environment. In this notebooks we use the following libraries:

- [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.
- [NumPy](http://www.numpy.org/) is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects.
- [urllib](https://docs.python.org/3/library/urllib.html) is an internal package that collects several modules for working with URLs.
- [zipfile](https://docs.python.org/3/library/zipfile.html) is an internal python module provides tools to create, read, write, append, and list a ZIP file.
- [tempfile](https://docs.python.org/3/library/tempfile.html) is an internal python package that creates temporary files and directories.
- [tqdm](https://github.com/tqdm/tqdm) is a smart progress meter that allows easy addition of a loop counter.
- [googleapiclient](https://pypi.org/project/google-api-python-client/) is the Python client library for Google's discovery based APIs. These client libraries are officially supported by Google.
- [oauth2client](https://pypi.org/project/oauth2client/) is a client library for OAuth 2.0, which is used to access the users Google Earth Engine account.
- [earthengine](https://pypi.org/project/earthengine-api/) allows developers to interact with Google Earth Engine using the Python programming language.

In [25]:
#Setup Environment
from pathlib import Path
import urllib
import zipfile
from tqdm.auto import tqdm
import shutil

import numpy as np
from osgeo import gdal
from osgeo import osr
import pyproj
from oauth2client import crypt

from ipyfilechooser import FileChooser

import googleapiclient
import ee

# 1. Define convenience functions

Here we define some functions for later convenience.
    
- **built_vrt** generate a virtual raster file using GDAL.
- **bounding_box_to_string** given west, east, south, north bounds, return a string e.g. 'E084_N025_E085_N024'
- **coordinate_to_string** return string from given lat/lon coordinates. e.g. 'E084_N025'
- **reproject** reprojects a given vector file to another coordinate reference system (CRS).
- **download_tile_from_ge** download a single tile from Google Earth Engine, not to exceed 32MB in size.
- **download_from_ge** calculates number of tiles needed given an area, downloads and stitches.
- **estimate_size** estimates tile size with SRTM-1arcsec format and sampling assumptions.    
- **gdal_get_geotransform** returns the geotransform of the dataset using GDAL.
- **gdal_get_projection** returns the spatial reference system in wkt, proj4, or epsg formats.
- **gdal_get_WESN** returns rectangle bounding box coordinates: west, east, south and north.
- **numel** returns number of elements for a wide range of data types.  
- **PathSelector** displays a file tree to easily browse and select a file.
- **transform_point** transforms a point coordinate to another coordinate reference system (CRS).
- **TqdmUpTo** callback function for TQDM counter used in warp function.
- **warp** a raster file into a new coordinate system using GDAL. Can also be used to combine multiple tiles into a single file.

In [26]:
#Define Constants and Functions

google_api_download_limit=33554432
def build_vrt(filename, input_file_list, targetAlignedPixels=True, separate=False, resampleAlg='near', resolution='highest'):
    filename = str(filename)
    input_file_list = [str(i) for i in input_file_list]
    
    vrt_options = gdal.BuildVRTOptions(resampleAlg=resampleAlg, resolution=resolution, separate=separate, targetAlignedPixels=targetAlignedPixels)
    ds=gdal.BuildVRT(filename, input_file_list, options=vrt_options)
    ds.FlushCache()

    
def bounding_box_to_string(w, e, s, n, factor=3):
    """Format a pair of angles lat, lon as a string:
    string = coordinate_to_string(lat, lon)
    """
    s1=coordinate_to_string(n,w, factor=factor)
    s2=coordinate_to_string(s,e, factor=factor)
    return '_'.join([s1,s2])
    
    
def coordinate_to_string(lat,lon, factor=4):
    """Format a pair of angles lat, lon as a string:
    string = coordinate_to_string(lat, lon)
    """
    def format_parts():
        fmt = f'{{prefix}}{{value:0>{str(factor+3)}d}}'
        for angle, directions in zip((lon, lat), ['EW', 'NS']):
            if angle >= 0:
                yield fmt.format(prefix=directions[0], value=int(angle*10**factor))
            else:
                yield fmt.format(prefix=directions[1], value=int(-angle*10**factor))
    return '_'.join(format_parts())


def download_tile_from_ge(W, E, S, N, 
                          collection_path=Path('users/gena/global-hand/hand-100'), 
                          prefix=None, fname=None, download_path=None, debug=False):
    if download_path is None:
        download_path = Path.cwd()
    else:
        download_path = Path(download_path)
    if prefix is None:
        if fname is None:
            prefix='ge'            
    else: #prefix is not None
        if fname is not None: # and fname is not None
            print('When fname is specified prefix is ignored.')

    geom = ee.Geometry.Polygon( [[E, S], [W, S], [W, N], [E, N], [E, S]] )    
    try:  
        #GENA HAND dataset is an image collection
        hand = ee.ImageCollection(str(collection_path))
        hand_clip = ee.ImageCollection(hand.filterBounds(geom))
        hand_mosaic = hand_clip.reduce(ee.Reducer.mean())
        ge_path = hand_mosaic.getDownloadUrl({
            'scale': 30,
            'crs': 'EPSG:4326',
            'region': geom,
            'maxPixels': 1e10
        })        
    except:
        #This allows for future expansion to other datasets like NASADEM
        #e.g.
        #download_from_ge(W,E,S,N, collection_path="NASA/NASADEM_HGT/001",download_path='/home/jovyan/nasadem_test/ge_nasadem.vrt', debug=True, keep_downloads=False)
        hand = ee.Image(collection_path)
        ge_path = hand.getDownloadUrl({
            'scale': 30,
            'crs': 'EPSG:4326',
            'region': geom,
            'maxPixels': 1e10
        })

    if fname is None:
        fname = Path(f'{prefix}_{W}W_{E}E_{S}S_{N}N.zip')
    if debug: print(f'Downloading {fname} from: {ge_path}')
    output_path = download_path/fname
    urllib.request.urlretrieve(ge_path, output_path)
    return output_path    


def download_from_ge(W, E, S, N, 
                     collection_path=Path('users/gena/global-hand/hand-100'), 
                     prefix=None, download_path=None, debug=False, 
                     keep_downloads=False, dstSRS=None):    
    #deal with input
    if prefix is None:
        prefix='ge'    
    if download_path is not None:
        download_path = Path(download_path)
    else:
        download_path = Path.cwd()/'ge_download.tif'
    download_folder = download_path.parent
    if not download_folder.exists():
        download_folder.mkdir()
    elif download_folder.is_file(): 
        print(f'Can not create download_folder: {download_folder}')
        print("There is already a file with thte same name.")
        raise ValueError

    #start download
    estimated_size = estimate_size(W, E, S, N) * 1.2 #overestimate a little. 
    tile_count = estimated_size / google_api_download_limit    
    if tile_count > 1: #32MB=32*1024*1024
        print(f'Area too large by a factor of: {tile_count}')        
    divider = int(np.ceil(np.sqrt(tile_count)) + 1)
    SN = np.linspace(S, N, divider)    
    WE = np.linspace(W, E, divider)
    if debug: print(f'SN: {SN}')
    if debug: print(f'WE: {WE}')
    s  = SN[0]
    w  = WE[0]
    zip_files = []
    print('Downloading...')
    for n in tqdm(SN[1:]):
        for e in tqdm(WE[1:]):
            if debug: print(f'w/e/s/n: {w}/{e}/{s}/{n}')
            #fname=prefix+f'_{w}W_{e}E_{s}S_{n}N.zip'
            fname = f"{'_'.join([prefix, bounding_box_to_string(w, e, s, n)])}.zip"
            if debug: print(f'download: {download_folder/fname}')
            if (download_folder/fname).is_file():
                zip_files.append(download_folder/fname)
            else:
                zip_files.append(download_tile_from_ge(w, e, s, n, 
                                                       collection_path=collection_path, 
                                                       fname=fname, 
                                                       download_path=download_folder, 
                                                       debug=debug))
            w = e.copy()
        s = n.copy()
        w = WE[0]
    if debug: print(zip_files)
    
    #start_splicing  
    print('Unzipping...')
    zip_contents = []
    extract_folders = [f.parent for f in zip_files]
    for f, extract_folder in zip(zip_files, extract_folders):        
        with zipfile.ZipFile(f, 'r') as zip_ref:  
            zip_contents.append(zip_ref.namelist())
            zip_ref.extractall(path=extract_folder)
    if debug: print(zip_contents)
    
    #convert zip_contents to list of full paths.
    vrt_contents=[ extract_folders[k]/zip_contents[k][0] for k in range(len(zip_files))]
    if debug: print(f'contents:{vrt_contents}')
    
    #combine with gdal
    warp([str(f) for f in vrt_contents], str(download_path), pixel_spacing=None,dstSRS=dstSRS) #vrt can leave too many files behind. Switching to warp. 

    #cleanup
    if debug or keep_downloads:
        print('Skipping cleanup in debug mode or when keep_downloads is set.')
        print(f'Files NOT deleted: {zip_files}')
        print(f'Folders NOT deleted: {extract_folders}')
    else:
        for f in zip_files:
            f.unlink()
        shutil.rmtree(vrt_contents[0].parent)
    print(f'Successfully generated:{download_path}')    
        
def estimate_size(W, E, S, N):
    pixels_per_deg = 3601
    bytes_per_pixel = 4
    return (E - W) * pixels_per_deg * (N - S) * pixels_per_deg * bytes_per_pixel    
    
def gdal_get_geotransform(filename):
    '''
    [top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution]=gdal_get_geotransform('/path/to/file')
    '''
    filename == str(filename)
    #http://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
    ds = gdal.Open(filename)
    return ds.GetGeoTransform()


def gdal_get_projection(filename, out_format='proj4'):
    """
    epsg_string=get_epsg(filename, out_format='proj4')
    """
    filename = str(filename)
    try:
      ds=gdal.Open(filename, gdal.GA_ReadOnly)
      srs=gdal.osr.SpatialReference()
      srs.ImportFromWkt(ds.GetProjectionRef())
    except: #I am not sure if this is working for datasets without a layer. The first try block should work mostly.
      ds=gdal.Open(filename, gdal.GA_ReadOnly)
      ly=ds.GetLayer()
      if ly is None:
        print(f"Can not read projection from file:{filename}")
        return None
      else:
        srs=ly.GetSpatialRef()
    if out_format.lower()=='proj4':
      return srs.ExportToProj4()
    elif out_format.lower()=='wkt':
      return srs.ExportToWkt()
    elif out_format.lower()=='epsg':
      crs=pyproj.crs.CRS.from_proj4(srs.ExportToProj4())
      return crs.to_epsg()

    
def gdal_get_WESN(filename):
    '''
    (minx,miny,maxx,maxy)=corners('/path/to/file')
    '''
    #http://stackoverflow.com/questions/2922532/obtain-latitude-and-longitude-from-a-geotiff-file
    filename = str(filename)
    ds = gdal.Open(filename)
    width = ds.RasterXSize
    height = ds.RasterYSize
    gt = ds.GetGeoTransform()
    minx = gt[0]
    miny = gt[3] + width*gt[4] + height*gt[5] 
    maxx = gt[0] + width*gt[1] + height*gt[2]
    maxy = gt[3] 
    return (minx, maxx, miny, maxy)


def numel(x):
    if isinstance(x, int):
      return 1
    elif isinstance(x, float):
      return 1
    elif isinstance(x, float):
      return 1
    elif isinstance(x, str):
      return 1
    elif isinstance(x, list) or isinstance(x, tuple):
      return len(x)
    elif isinstance(x, np.ndarray):
      return x.size
    else: 
      print('Unknown type {}.'.format(type(x)))
      return None

    
def transform_point(x, y, z, 
                    s_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs', 
                    t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'):
    '''
    transform_point(x,y,z,s_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs', t_srs='+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')
    
    Known Bugs: gdal transform may fail if a proj4 string can not be found for the EPSG or WKT formats. 
    '''    
    srs_cs = osr.SpatialReference()    
    if "EPSG" == s_srs[0:4]:    
      srs_cs.ImportFromEPSG(int(s_srs.split(':')[1]))
    elif "GEOCCS" == s_srs[0:6]:
      srs_cs.ImportFromWkt(s_srs)
    else:
      srs_cs.ImportFromProj4(s_srs)

    trs_cs = osr.SpatialReference()    
    if "EPSG" == t_srs[0:4]:    
      trs_cs.ImportFromEPSG(int(t_srs.split(':')[1]));
    elif "GEOCCS" == t_srs[0:6]:
      trs_cs.ImportFromWkt(t_srs)
    else:
      trs_cs.ImportFromProj4(t_srs)
    transform = osr.CoordinateTransformation(srs_cs,trs_cs) 
    
    if numel(x) > 1:
        return [transformPoint(x[k], y[k], z[k]) for k in range(numel(x))]
    else:
        try:
            return transform.TransformPoint((x,y,z))
        except: 
            return transform.TransformPoint(x,y,z)

        
class TqdmUpTo(tqdm): #Used in warp()
    """Provides `update_to(n)` which uses `tqdm.update(delta_n)`."""
    def update_to(self, b=1, bsize=1, tsize=None):
        """
        b  : int, optional
            Number of blocks transferred so far [default: 1].
        bsize  : int, optional
            Size of each block (in tqdm units) [default: 1].
        tsize  : int, optional
            Total size (in tqdm units). If [default: None] remains unchanged.
        """
        if tsize is not None:
            self.total = tsize
        return self.update(b * bsize - self.n)  # also sets self.n = b * bsize
    def callback(self, complete, message, data):
        percent = int(complete * 100)  # round to integer percent
        self.update_to(percent, tsize=100)
        
        
def warp(src_filename, dst_filename, pixel_spacing=0.00008333, xRes=None, yRes=None, resampleAlg='nearest', dstSRS="EPSG:4326", tps=False, rpc=False):
    if xRes is None and pixel_spacing:
        xRes = pixel_spacing
    if yRes is None and pixel_spacing:
        yRes = pixel_spacing
    t = TqdmUpTo()
    gwo = gdal.WarpOptions(xRes=xRes, yRes=yRes, resampleAlg=resampleAlg, dstSRS=dstSRS, callback=t.callback)

    gdal.Warp(dst_filename, src_filename, options=gwo)
    del t

## 2. Define input parameters

Please set coordinates, output file and other options.

In [27]:
# Define Input Parameters

## 1. Bounding Box for the Area of Interest ##
# Set same value (e.g. all zeros) to select bounding box using file. 

W = 0 #92.0#-96.5001618 #upper left -96.5 / -95.1 / 39.9 / 41.5
N = 0 #25.0#41.5002284
E = 0 #93.0#-95.1003219 #lower right
S = 0 #24.0#39.9001804

## 2. Output file name ##
print("Choose a file name for the HAND tif")
while True:
    output_file = Path.cwd()/f"HAND_files/{input()}.tif"
    if output_file == "":
        print("Please enter a valid directory name")
        continue
    else:
        output_file.parent.mkdir(parents=True, exist_ok=True)
        break

Choose a file name for the HAND tif


 HAND_TEST


## 3. Login to Google Earth Engine

If this is your first time logging in, you will need to execute a command on terminal.

In [28]:
#Login to Google Earth Engine
try:
    ee.Initialize()
except:
    print('In a terminal run: conda activate hydrosar && earthengine authenticate')

## 4. Select file if coordinates are set to zero

The file is used to define the bounding box. 

In [29]:
# Select gdal file if WESN is all zeros. 
if W == E or N == S:
    print("Choose a tif file to determine the extent on which to crop the HAND file:")
    # f = PathSelector('.')
    # display(f.accord)
    f = FileChooser('/home/jovyan/notebooks')
    display(f)
else:
    pass

Choose a tif file to determine the extent on which to crop the HAND file:


FileChooser(path='/home/jovyan/notebooks', filename='', title='', show_hidden=False, select_desc='Select', cha…

In [30]:
#Get W,E,S,N from file if needed. 
if W == E or N == S:
    gdal_file = Path(f.selected)   
    if gdal_file.exists():
        print(f'Selected file: {gdal_file}')
    else:
        print(f'Can not find file: {gdal_file}')
        raise ValueError
        
    W, E, S, N = gdal_get_WESN(gdal_file)
    epsg = gdal_get_projection(gdal_file, out_format='epsg')
    if epsg == "4326":
        pass
    else:
        srs = gdal_get_projection(gdal_file, out_format='proj4')
        W, N, h = transform_point(W, N, 0, s_srs=srs)
        E, S, h = transform_point(E, S, 0, s_srs=srs)
        W = round(W, 2)
        E = round(E, 2)
        S = round(S, 2)
        N = round(N, 2)
        del h # we don't use height
    # the epsg of the downloaded file will be 4326
    epsg = '4326'
    print(f'Bounding Box W/E/S/N: {W} / {E} / {S} / {N}')
else:
    epsg = '4326'

Selected file: /home/jovyan/notebooks/Hydrosar/SAR_Images/Cropped_Zone1_dem/Zone1_merged.tiff
Bounding Box W/E/S/N: 89.09 / 90.02 / 22.57 / 23.59


## 5. Download from Google Earth Engine

Depending on the area requested this could take a while.

In [31]:
# Download and stitch tiles 
download_from_ge(W, E, S, N, 
                 download_path=output_file, 
                 debug=1, keep_downloads=None, 
                 dstSRS=f"EPSG:{str(epsg)}")

Area too large by a factor of: 1.7596273711109014
SN: [22.57 23.08 23.59]
WE: [89.09  89.555 90.02 ]
Downloading...


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

w/e/s/n: 89.09/89.555/22.57/23.08
download: /home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089090_N023080_E089555_N022570.zip
w/e/s/n: 89.555/90.02/22.57/23.08
download: /home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089555_N023080_E090020_N022570.zip


  0%|          | 0/2 [00:00<?, ?it/s]

w/e/s/n: 89.09/89.555/23.08/23.59
download: /home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089090_N023590_E089555_N023080.zip
w/e/s/n: 89.555/90.02/23.08/23.59
download: /home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089555_N023590_E090020_N023080.zip
[PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089090_N023080_E089555_N022570.zip'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089555_N023080_E090020_N022570.zip'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089090_N023590_E089555_N023080.zip'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089555_N023590_E090020_N023080.zip')]
Unzipping...
[['download.b1_mean.tif'], ['download.b1_mean.tif'], ['download.b1_mean.tif'], ['download.b1_mean.tif']]
contents:[PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/download.b1_mean.tif'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/download.b1_mean.tif'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/download.b1_mean.tif'), P

0it [00:00, ?it/s]

Skipping cleanup in debug mode or when keep_downloads is set.
Files NOT deleted: [PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089090_N023080_E089555_N022570.zip'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089555_N023080_E090020_N022570.zip'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089090_N023590_E089555_N023080.zip'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files/ge_E089555_N023590_E090020_N023080.zip')]
Folders NOT deleted: [PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files'), PosixPath('/home/jovyan/notebooks/Hydrosar/HAND_files')]
Successfully generated:/home/jovyan/notebooks/Hydrosar/HAND_files/HAND_TEST.tif


---

*Version 0.1.0 - Batu Osmanoglu*
*Version 0.1.1 - Batu Osmanoglu*
*Version 0.1.2 - Alex Lewandowski & Rui Kawahara*
    
*Change Log:*
- *2021/11/22: v0.1.2 Alex Lewandowski & Rui Kawahara*
    - *Feat: Use url-widget to access url via js, needed for jupyterLab*
    - *Feat: Use ipyfilechooseripyfilechooser to shorten notebook*
    - *Feat: Change html to Markdown for better rendering on GitHub*
    - *Feat: os -> pathlib*
- *2021/01/24: v0.1.1*
    - *Feat: Minor organization change, and added comments*
- *2021/01/13: v0.1.0*
- ***Initial version:** 2021/04/016*